<a href="https://colab.research.google.com/github/FiljohnDelinia/CPE-201L-DSA-2-A/blob/main/CLASSROOM_MANAGEMENT_SYSTEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tkinter as tk
from tkinter import messagebox, ttk
from tkcalendar import DateEntry
import os
import pandas as pd
from datetime import datetime, timedelta
from typing import List, Optional, Tuple
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

EXCEL_FILE = "booking_requests.xlsx"
EXPECTED_COLUMNS = ["Name", "Room Type", "Room Detail", "Date", "Start Time", "End Time", "Units"]

LECTURE_ROOMS = ["101", "102", "103", "104", "201", "202", "203", "204"]
LAB_ROOMS = [
   "Computer Laboratory", "Electrical Laboratory", "Electronics Laboratory",
   "Physics Laboratory", "Chemistry Laboratory", "IE Laboratory"
]

TIME_SLOTS = ["00", "15", "30", "45"]
HOURS = [str(h) for h in range(1, 13)]

class TimeUtils:
   @staticmethod
   def time_to_minutes(time_str: str) -> int:
       try:
           time_obj = datetime.strptime(time_str, "%I:%M %p")
           return time_obj.hour * 60 + time_obj.minute
       except ValueError as e:
           logger.error(f"Invalid time format: {time_str}")
           raise ValueError(f"Invalid time format: {time_str}") from e

   @staticmethod
   def minutes_to_time(minutes: int) -> str:
       hour = minutes // 60
       minute = minutes % 60
       ampm = "AM" if hour < 12 else "PM"
       hour = hour % 12 or 12
       return f"{hour}:{minute:02d} {ampm}"

   @staticmethod
   def validate_time_range(start_time: str, end_time: str) -> bool:
       start_min = TimeUtils.time_to_minutes(start_time)
       end_min = TimeUtils.time_to_minutes(end_time)
       return start_min < end_min

   @staticmethod
   def get_current_datetime() -> Tuple[str, int]:
       now = datetime.now()
       current_date = now.strftime("%m/%d/%y")
       current_minutes = now.hour * 60 + now.minute
       return current_date, current_minutes

class BookingRequest:
   def __init__(self, user: str, room_type: str, room_detail: str,
                date: str, start_time: str, end_time: str, units: int = 0):
       self.user = user
       self.room_type = room_type
       self.room_detail = room_detail
       self.date = date
       self.start_time = start_time
       self.end_time = end_time
       self.units = units
       self.timestamp = datetime.now()
       self._validate()

   def _validate(self):
       if not all([self.user, self.room_type, self.room_detail, self.date, self.start_time, self.end_time]):
           raise ValueError("All fields must be filled")

       if not TimeUtils.validate_time_range(self.start_time, self.end_time):
           raise ValueError("End time must be after start time")

       if self.room_type == "Lecture" and self.room_detail not in LECTURE_ROOMS:
           raise ValueError(f"Invalid lecture room: {self.room_detail}")
       elif self.room_type == "Laboratory" and self.room_detail not in LAB_ROOMS:
           raise ValueError(f"Invalid laboratory: {self.room_detail}")

   def get_duration_minutes(self) -> int:
       start_min = TimeUtils.time_to_minutes(self.start_time)
       end_min = TimeUtils.time_to_minutes(self.end_time)
       return end_min - start_min

   def is_expired(self) -> bool:
       current_date, current_minutes = TimeUtils.get_current_datetime()
       if self.date != current_date:
           return self.date < current_date

       end_min = TimeUtils.time_to_minutes(self.end_time)
       return current_minutes >= end_min

   def __str__(self):
       return f"{self.user} | {self.room_type} ({self.room_detail}) | {self.date} | {self.start_time} – {self.end_time} | {self.units} units"

class RoomBookingQueue:
   def __init__(self):
       self.queue: List[BookingRequest] = []
       self.operation_log = []

   def enqueue(self, request: BookingRequest) -> bool:
       if self.has_conflict(request):
           return False

       self.queue.append(request)
       self.operation_log.append(
           f"ENQUEUE: {request.user} - {request.room_detail} at {request.timestamp.strftime('%H:%M:%S')}")

       logger.info(f"Booking added (FIFO): {request}")
       logger.info(f"Queue size: {len(self.queue)}")
       return True

   def dequeue(self) -> Optional[BookingRequest]:
       if not self.queue:
           return None

       request = self.queue.pop(0)
       self.operation_log.append(f"DEQUEUE: {request.user} - {request.room_detail}")

       logger.info(f"Booking removed (FIFO): {request}")
       logger.info(f"Queue size: {len(self.queue)}")
       return request

   def peek(self) -> Optional[BookingRequest]:
       return self.queue[0] if self.queue else None

   def get_all_requests(self) -> List[BookingRequest]:
       return self.queue.copy()

   def delete_request(self, index: int) -> bool:
       if 0 <= index < len(self.queue):
           removed_request = self.queue[index]
           del self.queue[index]
           self.operation_log.append(f"DELETE: {removed_request.user} - {removed_request.room_detail} (index {index})")
           logger.info(f"Booking deleted: {removed_request}")
           return True
       return False

   def get_operation_log(self) -> List[str]:
       return self.operation_log.copy()

   def clear_operation_log(self):
       self.operation_log.clear()

   def has_conflict(self, request: BookingRequest) -> bool:
       new_start_min = TimeUtils.time_to_minutes(request.start_time)
       new_end_min = TimeUtils.time_to_minutes(request.end_time)

       for existing_req in self.queue:
           if (existing_req.room_type == request.room_type and
                   existing_req.date == request.date and
                   existing_req.room_detail == request.room_detail):

               existing_start = TimeUtils.time_to_minutes(existing_req.start_time)
               existing_end = TimeUtils.time_to_minutes(existing_req.end_time)

               if new_start_min < existing_end and new_end_min > existing_start:
                   return True
       return False

   def suggest_next_available(self, room_type: str, date: str,
                              duration_minutes: int, room_detail: str) -> Tuple[Optional[str], Optional[str]]:
       relevant_bookings = [
           req for req in self.queue
           if req.room_type == room_type and req.date == date and req.room_detail == room_detail
       ]

       booked_slots = []
       for req in relevant_bookings:
           start = TimeUtils.time_to_minutes(req.start_time)
           end = TimeUtils.time_to_minutes(req.end_time)
           booked_slots.append((start, end))

       booked_slots.sort()

       start_of_day, end_of_day = 8 * 60, 18 * 60
       current = start_of_day

       for start, end in booked_slots:
           if start - current >= duration_minutes:
               return (TimeUtils.minutes_to_time(current),
                       TimeUtils.minutes_to_time(current + duration_minutes))
           current = max(current, end)

       if end_of_day - current >= duration_minutes:
           return (TimeUtils.minutes_to_time(current),
                   TimeUtils.minutes_to_time(current + duration_minutes))

       return None, None

   def remove_expired_requests(self) -> int:
       before = len(self.queue)
       expired_requests = [req for req in self.queue if req.is_expired()]

       self.queue = [req for req in self.queue if not req.is_expired()]

       after = len(self.queue)
       removed_count = before - after

       if removed_count > 0:
           for req in expired_requests:
               self.operation_log.append(f"AUTO-DEQUEUE (expired): {req.user} - {req.room_detail}")
           logger.info(f"Removed {removed_count} expired bookings using FIFO")

       return removed_count

class ExcelManager:
   def __init__(self, filename: str):
       self.filename = filename

   def load_requests(self) -> List[BookingRequest]:
       if not os.path.exists(self.filename):
           return []

       try:
           df = pd.read_excel(self.filename)
           if not all(col in df.columns for col in EXPECTED_COLUMNS):
               logger.warning("Excel file missing expected columns")
               return []

           requests = []
           for _, row in df.iterrows():
               try:
                   req = BookingRequest(
                       user=str(row["Name"]) if pd.notna(row["Name"]) else "",
                       room_type=str(row["Room Type"]) if pd.notna(row["Room Type"]) else "",
                       room_detail=str(row["Room Detail"]) if pd.notna(row["Room Detail"]) else "",
                       date=str(row["Date"]) if pd.notna(row["Date"]) else "",
                       start_time=str(row["Start Time"]) if pd.notna(row["Start Time"]) else "",
                       end_time=str(row["End Time"]) if pd.notna(row["End Time"]) else "",
                       units=int(row["Units"]) if pd.notna(row["Units"]) else 0
                   )
                   requests.append(req)
               except (ValueError, KeyError) as e:
                   logger.warning(f"Skipping invalid row {_}: {e}")
                   continue

           logger.info(f"Loaded {len(requests)} bookings from Excel")
           return requests

       except Exception as e:
           logger.error(f"Error loading Excel file: {e}")
           return []

   def save_requests(self, requests: List[BookingRequest]) -> bool:
       try:
           data = []
           for req in requests:
               data.append([
                   req.user, req.room_type, req.room_detail,
                   req.date, req.start_time, req.end_time, req.units
               ])

           df = pd.DataFrame(data, columns=EXPECTED_COLUMNS)
           df.to_excel(self.filename, index=False)
           logger.info(f"Saved {len(requests)} bookings to Excel")
           return True

       except Exception as e:
           logger.error(f"Error saving to Excel: {e}")
           return False

class BookingApp:
   def __init__(self, root):
       self.root = root
       self.root.title("Classroom Management System - FIFO Demonstration")

       self.booking_queue = RoomBookingQueue()
       self.excel_manager = ExcelManager(EXCEL_FILE)

       self.setup_ui()
       self.load_initial_data()
       self.schedule_next_expiry()

   def setup_ui(self):
       main_frame = ttk.Frame(self.root, padding="10")
       main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

       input_frame = ttk.LabelFrame(main_frame, text="Booking Details", padding="5")
       input_frame.grid(row=0, column=0, columnspan=2, sticky=(tk.W, tk.E), pady=(0, 10))

       ttk.Label(input_frame, text="Year/Section").grid(row=0, column=0, sticky=tk.W, pady=2)
       self.entry_name = ttk.Entry(input_frame)
       self.entry_name.grid(row=0, column=1, sticky=(tk.W, tk.E), pady=2, padx=(5, 0))

       ttk.Label(input_frame, text="Room Type").grid(row=1, column=0, sticky=tk.W, pady=2)
       self.room_var = tk.StringVar(value="Lecture")
       room_type_dropdown = ttk.Combobox(input_frame, textvariable=self.room_var,
                                         values=["Lecture", "Laboratory"], state="readonly")
       room_type_dropdown.grid(row=1, column=1, sticky=(tk.W, tk.E), pady=2, padx=(5, 0))

       ttk.Label(input_frame, text="Room Number / Lab Name").grid(row=1, column=2, sticky=tk.W, pady=2)
       self.room_detail_var = tk.StringVar()
       self.room_detail_dropdown = ttk.Combobox(input_frame, textvariable=self.room_detail_var, state="readonly")
       self.room_detail_dropdown.grid(row=1, column=3, sticky=(tk.W, tk.E), pady=2, padx=(5, 0))

       self.room_var.trace_add("write", self.update_room_detail_options)
       self.update_room_detail_options()

       ttk.Label(input_frame, text="Date").grid(row=2, column=0, sticky=tk.W, pady=2)
       self.date_entry = DateEntry(input_frame, width=12, background='darkblue',
                                   foreground='white', borderwidth=2, showweeknumbers=False)
       self.date_entry.grid(row=2, column=1, sticky=tk.W, pady=2, padx=(5, 0))

       ttk.Label(input_frame, text="Start Time").grid(row=3, column=0, sticky=tk.W, pady=2)
       self.start_hour = tk.StringVar(value="9")
       self.start_minute = tk.StringVar(value="00")
       self.start_ampm = tk.StringVar(value="AM")
       self._create_time_inputs(input_frame, self.start_hour, self.start_minute, self.start_ampm, 3, 1)

       unit_frame = ttk.Frame(input_frame)
       unit_frame.grid(row=4, column=0, columnspan=4, pady=5)

       ttk.Label(unit_frame, text="Subject Units:").grid(row=0, column=0, sticky=tk.W)

       self.unit_var = tk.StringVar(value="1")
       ttk.Radiobutton(unit_frame, text="1 Unit", variable=self.unit_var, value="1").grid(row=0, column=1, sticky=tk.W, padx=5)
       ttk.Radiobutton(unit_frame, text="2 Units", variable=self.unit_var, value="2").grid(row=0, column=2, sticky=tk.W, padx=5)
       ttk.Radiobutton(unit_frame, text="3 Units", variable=self.unit_var, value="3").grid(row=0, column=3, sticky=tk.W, padx=5)
       ttk.Radiobutton(unit_frame, text="4 Units", variable=self.unit_var, value="4").grid(row=0, column=4, sticky=tk.W, padx=5)

       self.unit_info_label = ttk.Label(unit_frame, text="Lecture: 1 unit = 1 hour | Laboratory: 1 unit = 3 hours",
                                       font=("Arial", 9, "italic"))
       self.unit_info_label.grid(row=1, column=0, columnspan=5, sticky=tk.W, pady=(5, 0))

       self.calculate_time_button = ttk.Button(unit_frame, text="Calculate Time", command=self.calculate_time)
       self.calculate_time_button.grid(row=2, column=0, columnspan=5, pady=5)

       self.time_result_label = ttk.Label(unit_frame, text="", foreground="blue")
       self.time_result_label.grid(row=3, column=0, columnspan=5, sticky=tk.W)

       button_frame = ttk.Frame(input_frame)
       button_frame.grid(row=5, column=0, columnspan=4, pady=10)

       ttk.Button(button_frame, text="Submit Request", command=self.submit_request).pack(side=tk.LEFT, padx=5)
       ttk.Button(button_frame, text="Check Availability", command=self.view_availability).pack(side=tk.LEFT, padx=5)

       list_frame = ttk.LabelFrame(main_frame, text="Pending Requests (FIFO Order)", padding="5")
       list_frame.grid(row=1, column=0, columnspan=2, sticky=(tk.W, tk.E, tk.N, tk.S), pady=(0, 10))

       self.listbox_requests = tk.Listbox(list_frame, width=80, height=15)
       self.listbox_requests.grid(row=0, column=0, columnspan=2, sticky=(tk.W, tk.E, tk.N, tk.S))

       scrollbar = ttk.Scrollbar(list_frame, command=self.listbox_requests.yview)
       scrollbar.grid(row=0, column=2, sticky='ns')
       self.listbox_requests.config(yscrollcommand=scrollbar.set)

       action_frame = ttk.Frame(list_frame)
       action_frame.grid(row=1, column=0, columnspan=3, pady=5)

       ttk.Button(action_frame, text="Delete Selected", command=self.delete_selected_request).pack(side=tk.LEFT, padx=5)
       ttk.Button(action_frame, text="Export to Excel", command=self.export_to_excel).pack(side=tk.LEFT, padx=5)
       ttk.Button(action_frame, text="Refresh", command=self.update_queue_display).pack(side=tk.LEFT, padx=5)

       fifo_frame = ttk.LabelFrame(main_frame, text="FIFO Operations Demo", padding="5")
       fifo_frame.grid(row=2, column=0, columnspan=2, sticky=(tk.W, tk.E), pady=(0, 10))

       ttk.Button(fifo_frame, text="Process Next Request (Dequeue)",
                  command=self.process_next_request).pack(side=tk.LEFT, padx=5)
       ttk.Button(fifo_frame, text="View Next Request (Peek)",
                  command=self.view_next_request).pack(side=tk.LEFT, padx=5)
       ttk.Button(fifo_frame, text="Show FIFO Operation Log",
                  command=self.show_fifo_log).pack(side=tk.LEFT, padx=5)
       ttk.Button(fifo_frame, text="Clear FIFO Log",
                  command=self.clear_fifo_log).pack(side=tk.LEFT, padx=5)

       self.queue_status = ttk.Label(fifo_frame, text="Queue: 0 requests")
       self.queue_status.pack(side=tk.RIGHT, padx=10)

       self.root.columnconfigure(0, weight=1)
       self.root.rowconfigure(0, weight=1)
       main_frame.columnconfigure(0, weight=1)
       main_frame.rowconfigure(1, weight=1)
       input_frame.columnconfigure(1, weight=1)
       input_frame.columnconfigure(3, weight=1)
       list_frame.columnconfigure(0, weight=1)
       list_frame.rowconfigure(0, weight=1)

   def _create_time_inputs(self, parent, hour_var, minute_var, ampm_var, row, column):
       frame = ttk.Frame(parent)
       frame.grid(row=row, column=column, sticky=tk.W)

       ttk.Combobox(frame, textvariable=hour_var, values=HOURS, width=3, state="readonly").pack(side=tk.LEFT)
       ttk.Combobox(frame, textvariable=minute_var, values=TIME_SLOTS, width=3, state="readonly").pack(side=tk.LEFT, padx=2)
       ttk.Combobox(frame, textvariable=ampm_var, values=["AM", "PM"], width=3, state="readonly").pack(side=tk.LEFT)

   def update_room_detail_options(self, *args):
       room_type = self.room_var.get()
       if room_type == "Lecture":
           self.room_detail_dropdown['values'] = LECTURE_ROOMS
       elif room_type == "Laboratory":
           self.room_detail_dropdown['values'] = LAB_ROOMS
       else:
           self.room_detail_dropdown['values'] = []
       self.room_detail_var.set("")

   def calculate_time(self):
       try:
           start_time = f"{self.start_hour.get()}:{self.start_minute.get()} {self.start_ampm.get()}"
           units = int(self.unit_var.get())
           room_type = self.room_var.get()

           start_min = TimeUtils.time_to_minutes(start_time)

           if room_type == "Lecture":
               duration_minutes = units * 60
           else:
               duration_minutes = units * 180

           end_min = start_min + duration_minutes
           end_time = TimeUtils.minutes_to_time(end_min)

           self.time_result_label.config(text=f"End Time: {end_time} (Duration: {duration_minutes//60} hours)")

       except Exception as e:
           messagebox.showerror("Error", f"Failed to calculate time: {str(e)}")

   def submit_request(self):
       try:
           name = self.entry_name.get().strip()
           room_type = self.room_var.get()
           room_detail = self.room_detail_var.get()
           date = self.date_entry.get()
           start_time = f"{self.start_hour.get()}:{self.start_minute.get()} {self.start_ampm.get()}"
           units = int(self.unit_var.get())

           if not all([name, room_type, room_detail]):
               messagebox.showerror("Error", "Please fill in all fields.")
               return

           start_min = TimeUtils.time_to_minutes(start_time)

           if room_type == "Lecture":
               duration_minutes = units * 60
           else:
               duration_minutes = units * 180

           end_min = start_min + duration_minutes
           end_time = TimeUtils.minutes_to_time(end_min)

           request = BookingRequest(name, room_type, room_detail, date, start_time, end_time, units)

           if self.booking_queue.has_conflict(request):
               messagebox.showwarning("Conflict", "This time slot is already booked.")

               suggested_start, suggested_end = self.booking_queue.suggest_next_available(
                   room_type, date, duration_minutes, room_detail
               )

               if suggested_start and suggested_end:
                   messagebox.showinfo("Suggestion",
                                       f"Next available slot: {suggested_start} - {suggested_end}")
               return

           if self.booking_queue.enqueue(request):
               self.update_queue_display()
               self.export_to_excel()
               self.schedule_next_expiry()

               messagebox.showinfo("Success",
                                   f"Booking request submitted successfully!\n"
                                   f"Added to queue position: {len(self.booking_queue.queue)}\n"
                                   f"Total requests in queue: {len(self.booking_queue.queue)}\n"
                                   f"Units: {units}\n"
                                   f"Time: {start_time} - {end_time}")

               self.entry_name.delete(0, tk.END)
               self.time_result_label.config(text="")
           else:
               messagebox.showerror("Error", "Failed to add booking request.")

       except ValueError as e:
           messagebox.showerror("Validation Error", str(e))
       except Exception as e:
           logger.error(f"Unexpected error in submit_request: {e}")
           messagebox.showerror("Error", "An unexpected error occurred.")

   def delete_selected_request(self):
       selected = self.listbox_requests.curselection()
       if not selected:
           messagebox.showinfo("Info", "Please select a request to delete.")
           return

       if messagebox.askyesno("Confirm Delete", "Are you sure you want to delete this booking?"):
           if self.booking_queue.delete_request(selected[0]):
               self.update_queue_display()
               self.export_to_excel()
               self.schedule_next_expiry()
           else:
               messagebox.showerror("Error", "Failed to delete the selected request.")

   def process_next_request(self):
       if not self.booking_queue.queue:
           messagebox.showinfo("FIFO Demo", "Queue is empty! Nothing to process.")
           return

       next_request = self.booking_queue.dequeue()
       if next_request:
           messagebox.showinfo("FIFO Dequeue",
                               f"Processed first request (FIFO):\n"
                               f"User: {next_request.user}\n"
                               f"Room: {next_request.room_detail}\n"
                               f"Time: {next_request.start_time} - {next_request.end_time}\n"
                               f"Units: {next_request.units}\n\n"
                               f"This request was removed from the front of the queue.")

           self.update_queue_display()
           self.export_to_excel()
           self.update_queue_status()

   def view_next_request(self):
       next_request = self.booking_queue.peek()
       if next_request:
           messagebox.showinfo("FIFO Peek",
                               f"Next request in queue (FIFO):\n"
                               f"User: {next_request.user}\n"
                               f"Room: {next_request.room_detail}\n"
                               f"Time: {next_request.start_time} - {next_request.end_time}\n"
                               f"Units: {next_request.units}\n\n"
                               f"This request is at the front but NOT removed.")
       else:
           messagebox.showinfo("FIFO Peek", "Queue is empty!")

   def show_fifo_log(self):
       log_entries = self.booking_queue.get_operation_log()
       if not log_entries:
           messagebox.showinfo("FIFO Operation Log", "No operations recorded yet.")
           return

       log_text = "FIFO Operation Log (First-In-First-Out):\n\n"
       log_text += "\n".join(log_entries)

       log_window = tk.Toplevel(self.root)
       log_window.title("FIFO Operation Log")
       log_window.geometry("600x400")

       text_widget = tk.Text(log_window, wrap=tk.WORD)
       scrollbar = ttk.Scrollbar(log_window, command=text_widget.yview)
       text_widget.config(yscrollcommand=scrollbar.set)

       text_widget.insert(tk.END, log_text)
       text_widget.config(state=tk.DISABLED)

       text_widget.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
       scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

   def clear_fifo_log(self):
       self.booking_queue.clear_operation_log()
       messagebox.showinfo("FIFO Log", "Operation log cleared.")

   def update_queue_status(self):
       count = len(self.booking_queue.queue)
       self.queue_status.config(text=f"Queue: {count} request(s)")

       if count == 0:
           self.queue_status.config(foreground="green")
       elif count < 5:
           self.queue_status.config(foreground="black")
       else:
           self.queue_status.config(foreground="red")

   def update_queue_display(self):
       self.listbox_requests.delete(0, tk.END)

       for i, req in enumerate(self.booking_queue.get_all_requests()):
           queue_pos = f"[{i + 1}] "
           self.listbox_requests.insert(tk.END, f"{queue_pos}{req}")

       self.update_queue_status()

   def load_initial_data(self):
       requests = self.excel_manager.load_requests()
       for req in requests:
           self.booking_queue.enqueue(req)
       self.update_queue_display()

   def export_to_excel(self):
       success = self.excel_manager.save_requests(self.booking_queue.get_all_requests())
       if not success:
           messagebox.showerror("Error", "Failed to export to Excel file.")

   def view_availability(self):
       room_type = self.room_var.get()
       room_detail = self.room_detail_var.get()
       date = self.date_entry.get()

       if not room_detail:
           messagebox.showinfo("Info", "Please select a room first.")
           return

       room_bookings = [
           req for req in self.booking_queue.get_all_requests()
           if req.room_type == room_type and req.room_detail == room_detail and req.date == date
       ]

       if not room_bookings:
           messagebox.showinfo("Availability", f"{room_detail} is available all day on {date}")
           return

       booking_info = [f"Bookings for {room_detail} on {date}:"]
       for req in sorted(room_bookings, key=lambda x: TimeUtils.time_to_minutes(x.start_time)):
           booking_info.append(f"  • {req.start_time} - {req.end_time}: {req.user} ({req.units} units)")

       messagebox.showinfo("Availability", "\n".join(booking_info))

   def schedule_next_expiry(self):
       if hasattr(self, '_expiry_job'):
           self.root.after_cancel(self._expiry_job)

       next_end = None
       current_date, current_minutes = TimeUtils.get_current_datetime()

       for req in self.booking_queue.get_all_requests():
           if req.date == current_date:
               end_min = TimeUtils.time_to_minutes(req.end_time)
               if end_min > current_minutes:
                   if not next_end or end_min < next_end:
                       next_end = end_min

       if next_end:
           delay_ms = (next_end - current_minutes) * 60 * 1000
           delay_ms = max(1000, delay_ms + 1000)

           logger.info(f"Next expiry check in {delay_ms / 1000 / 60:.1f} minutes")
           self._expiry_job = self.root.after(delay_ms, self.remove_expired_and_reschedule)
       else:
           self._expiry_job = self.root.after(60000, self.remove_expired_and_reschedule)

   def remove_expired_and_reschedule(self):
       removed_count = self.booking_queue.remove_expired_requests()
       if removed_count > 0:
           self.export_to_excel()
           self.update_queue_display()
           messagebox.showinfo("Booking Update", f"{removed_count} booking(s) expired and were removed.")

       self.schedule_next_expiry()

if __name__ == "__main__":
   try:
       root = tk.Tk()
       root.title("Classroom Booking System - FIFO Queue Demonstration")
       app = BookingApp(root)
       root.mainloop()
   except Exception as e:
       logger.critical(f"Application failed to start: {e}")
       messagebox.showerror("Fatal Error", f"The application failed to start:\n{e}")